In [37]:
# Import Numpy & PyTorch
import numpy as np
import torch
from sklearn.linear_model import LinearRegression

A tensor is a number, vector, matrix or any n-dimensional array.

## Problem Statement

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [38]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [39]:
# Target (apples)
targets = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [40]:
# Convert inputs and targets to tensors
input_tensor = torch.tensor(inputs)
print('Tensor Inputs  :  \n', input_tensor)
target_tensor = torch.tensor(targets)
print('\nTensor targets  :  \n', target_tensor)

Tensor Inputs  :  
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

Tensor targets  :  
 tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [41]:
def fit(x, w_t, b):
  return (x @ w_t) + b

In [42]:
# Weights and biases
weights = torch.randn(1, 3, requires_grad=True)
biases = torch.randn(1, requires_grad=True)
weights_transpose = torch.transpose(weights, 0, 1)

print("Weights  :  ",weights)
print("\nBiases  :  ",biases)
print("\nWeights_transpose  :  \n",weights_transpose)

Weights  :   tensor([[-1.1656,  0.4522,  1.0506]], requires_grad=True)

Biases  :   tensor([-0.0503], requires_grad=True)

Weights_transpose  :  
 tensor([[-1.1656],
        [ 0.4522],
        [ 1.0506]], grad_fn=<TransposeBackward0>)


In [43]:
#predictions
pred = fit(input_tensor, weights_transpose, biases)
print(pred)

print("Target tensor :- ",target_tensor)

tensor([[ -9.6675],
        [  0.9100],
        [ 20.0695],
        [-60.6260],
        [ 36.4743]], grad_fn=<AddBackward0>)
Target tensor :-  tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


**Compare with** **targets**

In [44]:
print(target_tensor)

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [45]:
def mse(pred, target):
  difference = pred - target
  return torch.sum(difference * difference) / difference.numel()

In [46]:
def model(x, w):
    return x @ w.T

In [47]:
#compute loss
loss = mse(pred, target_tensor)
print('Loss   :  ', loss)


#compute gradient
loss.backward()

Loss   :   tensor(6353.3179, grad_fn=<DivBackward0>)


In [48]:
#The gradients are stored in the .grad property of the respective tensors.
# Gradients for weights
print(weights, end="\n-------------\n")
print(weights.grad)

tensor([[-1.1656,  0.4522,  1.0506]], requires_grad=True)
-------------
tensor([[-13482.7969, -13857.4863,  -8560.5566]])


In [49]:
weights.grad.zero_()
biases.grad.zero_()
print(weights.grad, end="\n-------\n")
print(biases.grad)

tensor([[0., 0., 0.]])
-------
tensor([0.])


**Adjust weights and biases using gradient descent**


We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the 5. gradient
6. Reset the gradients to zero

In [50]:
#prediction
prediction = fit(input_tensor, weights.t(), biases)
print(prediction)

tensor([[ -9.6675],
        [  0.9100],
        [ 20.0695],
        [-60.6260],
        [ 36.4743]], grad_fn=<AddBackward0>)


In [51]:
#loss
loss = mse(prediction, target_tensor)
print(loss)

tensor(6353.3179, grad_fn=<DivBackward0>)


In [52]:
#compute gradients
loss.backward()

In [53]:
# Adjust weights & reset gradients
with torch.no_grad():
    weights -= weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    weights.grad.zero_()
    biases.grad.zero_()
print(weights)

tensor([[-1.0308,  0.5908,  1.1362]], requires_grad=True)


In [54]:
# Calculate loss
prediction = fit(input_tensor, weights.t(), biases)
loss = mse(prediction, target_tensor)
print(loss)

tensor(2682.4985, grad_fn=<DivBackward0>)


**Train for multiple epochs**


To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [55]:
# Train for 100 epochs
for i in range(100):
    prediction = fit(input_tensor, weights.t(), biases)
    loss = mse(prediction, target_tensor)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

In [56]:
# Calculate loss
prediction = fit(input_tensor, weights.t(), biases)
loss = mse(prediction, target_tensor)
print(loss)

tensor(32.7809, grad_fn=<DivBackward0>)


In [57]:
#prediction
print(prediction)

tensor([[ 55.9762],
        [ 85.2343],
        [113.7190],
        [ 14.6018],
        [110.9594]], grad_fn=<AddBackward0>)


In [58]:
#tensor targets
print(target_tensor)

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])
